In [1]:
from torchvision.models import resnet18
# import TORCHVISION.MODELS.RESNET
# https://blog.paperspace.com/writing-cnns-from-scratch-in-pytorch/
import os
import ast
from PIL import Image
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import torchvision
import torch.nn.functional as F
import torch
import torch.optim as optim
import helper
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data.sampler import SubsetRandomSampler
# import torch.utils.data as data
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torchsummary import summary
from imblearn.under_sampling import RandomUnderSampler

num_classes = 12
learning_rate = 0.001
num_epochs = 2
batch_size = 64
patience = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
ROOT_DIR = "/workspace/resnet/"
ROOT_DATA_DIR = "/workspace/resnet/data/raw"
ARCHIVE_DIR = os.path.join(ROOT_DATA_DIR, "archive.zip")
DATA_DIR = os.path.join(ROOT_DATA_DIR)#, "labelme-12-50k")
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TRAIN_ANNOT_PATH = os.path.join(TRAIN_DIR, "annotation.txt")
TEST_DIR = os.path.join(DATA_DIR, "test")
TEST_ANNOT_PATH = os.path.join(TEST_DIR, "annotation.txt")
CLASSES_TXT_PATH = os.path.join(DATA_DIR, "classes.txt")
INTERIM_DATA_DIR = "/workspace/resnet/data/interim/"
MODEL_DIR = os.path.join(ROOT_DIR, "models")
CNN_PATH = os.path.join(MODEL_DIR, "2.2_resnet-balanced")

# import zipfile
# with zipfile.ZipFile(ARCHIVE_DIR, 'r') as zip_ref:
#     zip_ref.extractall(ROOT_DATA_DIR)

cpu


In [2]:
# resnet.eval()

In [3]:
class MyResModel(nn.Module):
    def __init__(self):
        super(MyResModel, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(1000,12)
        )

    def forward(self, x):
        return self.classifier(x)

In [4]:
resnet = resnet18(pretrained=True)
resnet.eval()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
n_feats = resnet.fc.in_features
resnet.fc = nn.Linear(n_feats, 12)#MyResModel()

for name, param in resnet.named_parameters():
#     print(name)
    if 'fc' not in name:
        print(name, param.requires_grad)
        param.requires_grad=False
# for param in resnet.parameters():
#     param.requires_grad_(False)

conv1.weight True
bn1.weight True
bn1.bias True
layer1.0.conv1.weight True
layer1.0.bn1.weight True
layer1.0.bn1.bias True
layer1.0.conv2.weight True
layer1.0.bn2.weight True
layer1.0.bn2.bias True
layer1.1.conv1.weight True
layer1.1.bn1.weight True
layer1.1.bn1.bias True
layer1.1.conv2.weight True
layer1.1.bn2.weight True
layer1.1.bn2.bias True
layer2.0.conv1.weight True
layer2.0.bn1.weight True
layer2.0.bn1.bias True
layer2.0.conv2.weight True
layer2.0.bn2.weight True
layer2.0.bn2.bias True
layer2.0.downsample.0.weight True
layer2.0.downsample.1.weight True
layer2.0.downsample.1.bias True
layer2.1.conv1.weight True
layer2.1.bn1.weight True
layer2.1.bn1.bias True
layer2.1.conv2.weight True
layer2.1.bn2.weight True
layer2.1.bn2.bias True
layer3.0.conv1.weight True
layer3.0.bn1.weight True
layer3.0.bn1.bias True
layer3.0.conv2.weight True
layer3.0.bn2.weight True
layer3.0.bn2.bias True
layer3.0.downsample.0.weight True
layer3.0.downsample.1.weight True
layer3.0.downsample.1.bias True
la

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [7]:
train_df_balanced = pd.read_csv(os.path.join(INTERIM_DATA_DIR,"balanced-train-40000.csv"))
test_df = pd.read_csv(os.path.join(INTERIM_DATA_DIR,"test-10000.csv"))

transform = transforms.Compose(
            [transforms.ToTensor(),
            transforms.Normalize((0.5,0.5,0.5),(0.5, 0.5, 0.5))]
            )
class MyDataset(Dataset):
    def __init__(self,annot_df,transform = None):
        self.annotations = annot_df
        self.transform = transform
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self,index):
        img = Image.open(self.annotations.img_path[index])
        y_label = torch.tensor(int(self.annotations.int_label[index]))
        if self.transform:
            image = transform(img)
        else:
            image = img
        return (image,y_label)
    
# train_data = MyDataset(train_df,transform = transform)
test_data = MyDataset(test_df,transform = transform)
train_data_balanced = MyDataset(train_df_balanced,transform = transform)


# train_loader = DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_data, batch_size = batch_size, shuffle = True)
train_loader_balanced = DataLoader(dataset = train_data_balanced, batch_size = batch_size, shuffle = True)

test_loader

In [8]:
model = resnet#ResNet(block,[3,4,6,3], 3, 12)
print(model)

model.train()
# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

# total_step = len(train_loader)
total_step = len(train_loader_balanced)

best_score = None
counter = 0

for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_loader_balanced):
#     for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    if best_score is None:
        best_score = loss
    else:
            # Check if val_loss improves or not.
        if loss < best_score:
                # val_loss improves, we update the latest best_score, 
                # and save the current model
            best_score = loss
            torch.save(model.state_dict(), CNN_PATH)
        else:
                # val_loss does not improve, we increase the counter, 
                # stop training if it exceeds the amount of patience
            counter += 1
            if counter >= patience:
                break
    if epoch == num_epochs:
        torch.save(model.state_dict(), CNN_PATH)

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Epoch [1/2], Loss: 1.2950
Epoch [1/2], Loss: 1.0949
Epoch [1/2], Loss: 1.1359
Epoch [1/2], Loss: 1.2055
Epoch [1/2], Loss: 1.0513
Epoch [1/2], Loss: 1.0424
Epoch [1/2], Loss: 0.9834
Epoch [1/2], Loss: 1.1770
Epoch [1/2], Loss: 1.2699
Epoch [1/2], Loss: 1.1618
Epoch [1/2], Loss: 1.0337
Epoch [1/2], Loss: 1.1966
Epoch [1/2], Loss: 1.0413
Epoch [1/2], Loss: 0.9823
Epoch [1/2], Loss: 0.9996
Epoch [1/2], Loss: 0.9906
Epoch [1/2], Loss: 1.1164
Epoch [1/2], Loss: 1.1101
Epoch [1/2], Loss: 0.9914
Epoch [1/2], Loss: 1.0365
Epoch [1/2], Loss: 1.2008
Epoch [1/2], Loss: 0.9877
Epoch [1/2], Loss: 0.9319
Epoch [1/2], Loss: 1.1854
Epoch [1/2], Loss: 0.9868
Epoch [1/2], Loss: 1.2335
Epoch [1/2], Loss: 1.0501
Epoch [1/2], Loss: 0.9213
Epoch [1/2], Loss: 0.8468
Epoch [1/2], Loss: 1.1262
Epoch [1/2], Loss: 1.1385
Epoch [1/2], Loss: 1.0712
Epoch [1/2], Loss: 1.1195
Epoch [1/2], Loss: 1.0349
Epoch [1/2], Loss: 1.0480
Epoch [1/2], Loss: 0.9671
Epoch [1/2], Loss: 1.1929
Epoch [1/2], Loss: 1.0867
Epoch [1/2],

Epoch [1/2], Loss: 0.8264
Epoch [1/2], Loss: 0.8275
Epoch [1/2], Loss: 0.9821
Epoch [1/2], Loss: 0.7920
Epoch [1/2], Loss: 0.9148
Epoch [1/2], Loss: 0.8381
Epoch [1/2], Loss: 0.8125
Epoch [1/2], Loss: 0.8649
Epoch [1/2], Loss: 0.8810
Epoch [1/2], Loss: 0.7200
Epoch [1/2], Loss: 0.7968
Epoch [1/2], Loss: 1.0750
Epoch [1/2], Loss: 0.7381
Epoch [1/2], Loss: 0.8323
Epoch [1/2], Loss: 0.7291
Epoch [1/2], Loss: 0.7744
Epoch [1/2], Loss: 0.7648
Epoch [1/2], Loss: 0.7578
Epoch [1/2], Loss: 0.7076
Epoch [1/2], Loss: 0.8505
Epoch [1/2], Loss: 1.1361
Epoch [1/2], Loss: 0.8480
Epoch [1/2], Loss: 0.7403
Epoch [1/2], Loss: 0.8777
Epoch [1/2], Loss: 0.7899
Epoch [1/2], Loss: 0.6724
Epoch [1/2], Loss: 0.8234
Epoch [1/2], Loss: 0.8335
Epoch [1/2], Loss: 0.7234
Epoch [1/2], Loss: 0.7786
Epoch [1/2], Loss: 0.5937
Epoch [1/2], Loss: 0.7854
Epoch [1/2], Loss: 0.8428
Epoch [1/2], Loss: 0.6615
Epoch [1/2], Loss: 0.8962
Epoch [1/2], Loss: 0.8272
Epoch [1/2], Loss: 0.7087
Epoch [1/2], Loss: 0.6982
Epoch [1/2],

Epoch [2/2], Loss: 0.7536
Epoch [2/2], Loss: 0.8482
Epoch [2/2], Loss: 0.6437
Epoch [2/2], Loss: 0.5828
Epoch [2/2], Loss: 0.8731
Epoch [2/2], Loss: 0.6917
Epoch [2/2], Loss: 0.7836
Epoch [2/2], Loss: 0.7003
Epoch [2/2], Loss: 0.8903
Epoch [2/2], Loss: 0.6854
Epoch [2/2], Loss: 0.5722
Epoch [2/2], Loss: 0.6832
Epoch [2/2], Loss: 0.8108
Epoch [2/2], Loss: 0.6701
Epoch [2/2], Loss: 0.8893
Epoch [2/2], Loss: 0.6145
Epoch [2/2], Loss: 0.8095
Epoch [2/2], Loss: 0.8395
Epoch [2/2], Loss: 0.7284
Epoch [2/2], Loss: 0.7829
Epoch [2/2], Loss: 0.5989
Epoch [2/2], Loss: 0.7931
Epoch [2/2], Loss: 0.6910
Epoch [2/2], Loss: 0.8012
Epoch [2/2], Loss: 0.6967
Epoch [2/2], Loss: 0.6562
Epoch [2/2], Loss: 0.8166
Epoch [2/2], Loss: 0.7977
Epoch [2/2], Loss: 0.7103
Epoch [2/2], Loss: 0.5422
Epoch [2/2], Loss: 0.7228
Epoch [2/2], Loss: 0.5390
Epoch [2/2], Loss: 0.7697
Epoch [2/2], Loss: 0.9358
Epoch [2/2], Loss: 0.8251
Epoch [2/2], Loss: 0.8245
Epoch [2/2], Loss: 0.8197
Epoch [2/2], Loss: 0.6622
Epoch [2/2],

Epoch [2/2], Loss: 0.6462
Epoch [2/2], Loss: 0.5632
Epoch [2/2], Loss: 0.7447
Epoch [2/2], Loss: 0.6056
Epoch [2/2], Loss: 0.7084
Epoch [2/2], Loss: 0.6453
Epoch [2/2], Loss: 0.6159
Epoch [2/2], Loss: 0.7807
Epoch [2/2], Loss: 0.5742
Epoch [2/2], Loss: 0.6026
Epoch [2/2], Loss: 0.6150
Epoch [2/2], Loss: 0.8924
Epoch [2/2], Loss: 0.6412
Epoch [2/2], Loss: 0.8039
Epoch [2/2], Loss: 0.5001
Epoch [2/2], Loss: 0.6470
Epoch [2/2], Loss: 0.6634
Epoch [2/2], Loss: 0.6757
Epoch [2/2], Loss: 0.6205
Epoch [2/2], Loss: 0.4657
Epoch [2/2], Loss: 0.7157
Epoch [2/2], Loss: 0.6368
Epoch [2/2], Loss: 0.7522
Epoch [2/2], Loss: 0.6964
Epoch [2/2], Loss: 0.6445
Epoch [2/2], Loss: 0.7308
Epoch [2/2], Loss: 0.7515
Epoch [2/2], Loss: 0.6628
Epoch [2/2], Loss: 0.7862
Epoch [2/2], Loss: 0.5535
Epoch [2/2], Loss: 0.8193
Epoch [2/2], Loss: 0.4798
Epoch [2/2], Loss: 0.5290
Epoch [2/2], Loss: 0.7329
Epoch [2/2], Loss: 0.5854
Epoch [2/2], Loss: 0.7196
Epoch [2/2], Loss: 0.4972
Epoch [2/2], Loss: 0.6335
Epoch [2/2],

In [9]:
train_df_balanced.int_label.value_counts()

0     3438
9     3422
7     3392
6     3325
11    3317
10    3312
8     3305
1     3303
2     3302
4     3297
5     3295
3     3292
Name: int_label, dtype: int64

In [14]:
print(model.state_dict())

OrderedDict([('conv1.weight', tensor([[[[-1.0419e-02, -6.1356e-03, -1.8098e-03,  ...,  5.6615e-02,
            1.7083e-02, -1.2694e-02],
          [ 1.1083e-02,  9.5276e-03, -1.0993e-01,  ..., -2.7124e-01,
           -1.2907e-01,  3.7424e-03],
          [-6.9434e-03,  5.9089e-02,  2.9548e-01,  ...,  5.1972e-01,
            2.5632e-01,  6.3573e-02],
          ...,
          [-2.7535e-02,  1.6045e-02,  7.2595e-02,  ..., -3.3285e-01,
           -4.2058e-01, -2.5781e-01],
          [ 3.0613e-02,  4.0960e-02,  6.2850e-02,  ...,  4.1384e-01,
            3.9359e-01,  1.6606e-01],
          [-1.3736e-02, -3.6746e-03, -2.4084e-02,  ..., -1.5070e-01,
           -8.2230e-02, -5.7828e-03]],

         [[-1.1397e-02, -2.6619e-02, -3.4641e-02,  ...,  3.2521e-02,
            6.6221e-04, -2.5743e-02],
          [ 4.5687e-02,  3.3603e-02, -1.0453e-01,  ..., -3.1253e-01,
           -1.6051e-01, -1.2826e-03],
          [-8.3730e-04,  9.8420e-02,  4.0210e-01,  ...,  7.0789e-01,
            3.6887e-01,  1.2

In [10]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader_balanced:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
#         prin
    
    print('Accuracy of the network on the {} train images: {} %'.format(total, 100 * correct / total))




Accuracy of the network on the 40000 train images: 80.3 %


In [13]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} test images: {} %'.format(total, 100 * correct / total))




Accuracy of the network on the 10000 test images: 67.74 %


In [ ]:
summary(CNN(12),(3, 256, 256))

In [ ]:
images, labels = next(iter(train_loader))
images.shape

In [ ]:
images[0].shape

In [ ]:
labels.shape

In [ ]:
torchvision.transforms.functional.get_image_size(images)

In [ ]:
print(test_df.str_label[1500])
Image.open(test_df.img_path[1500])
print(train_df.str_label[1000],train_df.img_path[1000])
Image.open(train_df.img_path[1000])

In [ ]:
print(train_df.str_label[1000],train_df.img_path[1000])
Image.open(train_df.img_path[1000])

In [ ]:
# transform = transforms.Compose(
#         [transforms.ToTensor(),
#         transforms.Normalize((0.5,0.5,0.5),(0.5, 0.5, 0.5))]
#     )

# test_data = datasets.ImageFolder('/workspace/resnet/data/raw/test/', transform=transform)
# test_data_loader  = data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4) 

# train_data = datasets.ImageFolder('/workspace/resnet/data/raw/train/', transform=transform)
# train_data_loader  = data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4) 

# train_data